In [4]:
#df = pd.read_csv(r'H:\MSC Data Science\Dissertation\data\PPG_FieldStudy\S1\S1_activity.csv')
#d = dtale.show(df)
#d
#共15个测试者，S5已知异常（可验证），取出一个测试者作为测试集，其他13个测试者的数据混合到一起，划分训练集和验证集（参考分心驾驶员检测）
#在胸部传感器信号数据中， ‘EDA’, ‘EMG’ and ‘Temp’无效，仅 ACC, ECG, RESP可用

#两种预测方式，1是把8秒平均心率视为每一秒都一样，去预测每秒，2是直接合并8秒的信号，预测8秒的平均心率，有准确的八秒平均也有每秒的
#异常检测
#先监测相关性，然后用滑窗来扩展变量数量
import numpy as np
import pandas as pd
import pickle

path_1 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S1/S1.pkl'
path_2 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S2/S2.pkl'
path_3 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S3/S3.pkl'
path_4 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S4/S4.pkl'
path_5 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S5/S5.pkl'
path_6 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S6/S6.pkl'
path_7 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S7/S7.pkl'
path_8 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S8/S8.pkl'
path_9 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S9/S9.pkl'
path_10 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S10/S10.pkl'
path_11 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S11/S11.pkl'
path_12 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S12/S12.pkl'
path_13 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S13/S13.pkl'
path_14 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S14/S14.pkl'
path_15 = 'H:/MSC Data Science/Dissertation/data/PPG_FieldStudy/S15/S15.pkl'

In [5]:
def load_data(file_path):
    S = pickle.load(open(file_path, 'rb'), encoding='latin1')#load pkl file as origin data
    
    signal = pd.DataFrame(S["signal"])
    chest = signal['chest']
    wrist = signal['wrist']
    
    chest_ACC = pd.DataFrame(chest['ACC'])
    chest_ACC = chest_ACC.iloc[::175, :]
    chest_ACC.columns = ["chest_ACC_x", "chest_ACC_y", "chest_ACC_z"]
    chest_ACC.reset_index(drop = True, inplace=True)
    
    chest_Resp = pd.DataFrame(chest['Resp'])
    chest_Resp = chest_Resp.iloc[::175, :]
    chest_Resp.columns = ["chest_Resp"]
    chest_Resp.reset_index(drop = True, inplace=True)
    
    chest_ECG = pd.DataFrame(chest['ECG'])
    chest_ECG = chest_ECG.iloc[::175, :]
    chest_ECG.reset_index(drop = True, inplace=True)
    
    data_chest = pd.concat([chest_ACC], sort=False)
    data_chest["Resp"] = chest_Resp
    data_chest["ECG"] = chest_ECG
    data_chest.reset_index(drop=True, inplace=True)

    wrist_ACC = pd.DataFrame(wrist['ACC'])
    wrist_ACC = wrist_ACC.iloc[::8, :]
    wrist_ACC.columns = ["wrist_ACC_x", "wrist_ACC_y", "wrist_ACC_z"]
    wrist_ACC.reset_index(drop = True, inplace=True)
    
    wrist_EDA = pd.DataFrame(wrist['EDA'])
    wrist_EDA.columns = ["wrist_EDA"]
    
    wrist_BVP = pd.DataFrame(wrist['BVP'])
    wrist_BVP = wrist_BVP.iloc[::16, :]
    wrist_BVP.columns = ["wrist_BVP"]
    wrist_BVP.reset_index(drop = True, inplace=True)
    
    wrist_TEMP = pd.DataFrame(wrist['TEMP'])
    wrist_TEMP.columns = ["wrist_TEMP"]
    
    data_wrist = pd.concat([wrist_ACC], sort=False)
    data_wrist["wrist_BVP"] = wrist_BVP
    data_wrist["wrist_TEMP"] = wrist_TEMP
    data_wrist.reset_index(drop = True, inplace=True)
    
    
    data_signal = data_chest.join(data_wrist)
    for k,v in S["questionnaire"].items() :
        data_signal[k] = v
    
    rpeaks = S['rpeaks']
    counted_rpeaks = []
    index = 0 # index of rpeak element
    time = 175 # time portion
    count = 0 # number of rpeaks

    while(index < len(rpeaks)):
        rpeak = rpeaks[index]

        if(rpeak > time): # Rpeak appears after the time portion
            counted_rpeaks.append(count)
            count = 0
            time += 175

        else:
            count += 1
            index += 1
    # The rpeaks will probably end before the time portion so we need to fill the last portions with 0
    if(len(counted_rpeaks) < np.size(data_signal, axis = 0)):
        while(len(counted_rpeaks) < np.size(data_signal, axis = 0)):
            counted_rpeaks.append(0)
    peaks = pd.DataFrame(counted_rpeaks)
    peaks.columns = ["Rpeaks"]
    data_signal = data_signal.join(peaks)
    
    activity = pd.DataFrame(S["activity"]).astype(int)
    activity.columns = ["Activity"]
    data_signal = data_signal.join(activity)
    
    label = pd.DataFrame(S["label"])

    label = pd.DataFrame(np.repeat(label.values,8,axis=0))
    label.columns = ["Label"]
    if(np.size(label, axis = 0) < np.size(activity, axis = 0)):
        mean = label.mean()
        while(np.size(label, axis = 0) < np.size(activity, axis = 0)):
            label = label.append(mean, ignore_index=True)
    
    data_signal = data_signal.join(label)
    
    data_signal['Subject'] = S["subject"]
    return data_signal

In [6]:
S1 = load_data(path_1)
S2 = load_data(path_2)
S3 = load_data(path_3)
S4 = load_data(path_4)
S5 = load_data(path_5)
S6 = load_data(path_6)
S7 = load_data(path_7)
S8 = load_data(path_8)
S9 = load_data(path_9)
S10 = load_data(path_10)
S11 = load_data(path_11)
S12 = load_data(path_12)
S13 = load_data(path_13)
S14 = load_data(path_14)
S15 = load_data(path_15)
print('load completed')

load completed


In [7]:
df_groups = [S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14,S15]
data = pd.concat(df_groups)
print(data)

       chest_ACC_x  chest_ACC_y  chest_ACC_z      Resp       ECG  wrist_ACC_x  \
0           0.8564      -0.0678      -0.3656  4.441833  0.015610    -0.765625   
1           0.8514      -0.0654      -0.3688  4.876709 -0.015747    -0.765625   
2           0.8514      -0.0662      -0.3708  3.340149 -0.008743    -0.765625   
3           0.8594      -0.0632      -0.3640  0.740051 -0.339523    -0.750000   
4           0.8510      -0.0674      -0.3694 -1.475525 -0.089905    -0.765625   
...            ...          ...          ...       ...       ...          ...   
31751       0.8878       0.0122      -0.2330  2.789307 -0.281845    -0.468750   
31752       0.8962      -0.0184      -0.2280  0.563049  0.075851    -0.609375   
31753       0.9010       0.0006      -0.2102 -1.075745  0.017166    -0.468750   
31754       0.8906      -0.0124      -0.2238 -2.117920 -0.304184    -0.375000   
31755       0.8888       0.0334      -0.2040 -2.513123 -0.062943    -0.359375   

       wrist_ACC_y  wrist_A

In [8]:
data.to_csv("dataset.csv", index=False)